In [7]:
"""
THIS CODE MAPS FRAME DATA TO GLOSS WORDS

It maps all the sentences to corresponing frame data:
For example: 
    HELLO	26	35
    POSS-s:S	43	46
    NAME	50	56
    IS	63	65
    fs-BRUCE	70	100
    
    is mapped to frame data between frames 26 to 100

The length of gloss words in a sentence is fixed to 10. It can be edited in code and sentence length can be modified.

The frame data used in this file is gathered present in:
    key_points_5.npy (If using 5 features) --> gathered using toJSON_PCA_5feat.ipynb
    key_points_42.npy (If using 42 features) --> gathered using toJSON_PCA_m.ipynb
"""
import numpy as np
import pandas as pd

In [8]:
def get_sen_str_frm_gloss(gloss):
    st = ''
    for i in gloss:
        st = st + i[0] +' '
    return st

In [13]:
def get_start_end_index_sentence(gloss):
    start = int(gloss[0][1])
    end = int(gloss[-1][2])
    return [start, end]

In [14]:
def map_pca_sentence(pca_data, gloss_dir, start, end):
#     try:
    key = gloss_dir.replace('Gloss/','').replace('.txt','').replace('gloss','front_x264')
    
    for i in pca_data:
        for k in i:
            if k == key:
                pca_feat = i[k]
    
#     pca_feat = pca_data[0][key]

    data_size = pca_feat.shape[0]
#     print(data_size)

    if start > data_size:
        return []
    elif start <= data_size and  end > data_size: 
        return pca_feat[start:data_size,:]
    else:
        return pca_feat[start:end,:]
#     except Exception as e:
#         print('WHat is this? ',e)
#         pass

In [15]:
gloss_dir = 'Gloss/SIB01-story01-gloss.txt'

In [16]:
gloss_dir = 'Gloss/SIA02-story01-gloss.txt'

In [17]:
def read_gloss(direc):
    g_txt = open(direc)
    g_txt = g_txt.read().split('\n')
    gloss = []
    for i in g_txt:
        x = i.split('\t')
        if x[0] != '':
            gloss.append(x)
    return gloss


"""
You can edit the sentence_length in the below function to generate different mapping. (It can serve as data augmentation)
"""
def match_gloss_sentence(gloss_dir):
    gls = read_gloss(gloss_dir)
    
    pca_data = np.load('key_points_5.npy')
    
    # can change this value to any other
    sentenence_length = 10
    
    sentences_mapped = []
    
    sentence = []
    start = []
    end = []
    
    for i in range(0,len(gls)):
        if i%(sentenence_length) == 0:
            if sentence != []:
                sentences_mapped.append(sentence)
                sentence = []
        sentence.append(gls[i])
    
    sentences_mapped.append(sentence)
#     print(np.array(sentences_mapped))
    
    
    final_data = []
    
    for i in sentences_mapped:
        gls_sentence = get_sen_str_frm_gloss(i)
        x = get_start_end_index_sentence(i)
        temp = map_pca_sentence(pca_data,gloss_dir,x[0],x[1])
        if temp == []:
            pass
        else:
            pca_feat_sentence = temp
            final_data.append([temp, gls_sentence])
            

    return final_data
match_gloss_sentence(gloss_dir)

C:\Users\Sashank\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


[[array([[-2768.90608951, -1019.32967937,  -960.20901344, -1160.96258128,
          -1188.36433671],
         [-2727.262217  , -1094.5872145 ,  -906.52740894, -1173.71664271,
          -1186.76635802],
         [-2697.73472343, -1151.31877092,  -863.58526787, -1119.80875486,
          -1197.35909889],
         [-2762.96531258, -1145.65639588,  -834.95944433,  -976.43535962,
          -1217.07492103],
         [-2762.42973573, -1112.44622336,  -838.17098412,  -971.85706541,
          -1245.09869861],
         [-2671.34832556, -1298.83197279,  -771.81016373, -1070.60278938,
          -1192.68397695],
         [-2675.78883222, -1257.84281283,  -807.64603385, -1099.47517525,
          -1188.63022966],
         [-2803.11331923,  -982.31078643,  -954.36880907, -1059.2047111 ,
          -1243.54822819],
         [-2778.56771077,  -919.3867672 ,  -958.46722861, -1164.79262687,
          -1236.12748479],
         [-2865.07668022,  -913.54692022, -1057.18681328, -1195.10897224,
          -1235.2

In [18]:
loaded_data = np.load('key_points_5.npy')
sentence_frame_mapped_data = []
count = 0
for key in loaded_data:
    for k in key:
        try:
            f_n = k.strip('front_x264')
            gloss_d = 'Gloss/'+f_n+'gloss.txt'
#             print(gloss_d)
            
            sen_mapped = match_gloss_sentence(gloss_d)
#             print(type(sen_mapped))
            
            if count == 0:
                sentence_frame_mapped_data = sen_mapped
            else:
                sentence_frame_mapped_data = np.vstack((sentence_frame_mapped_data,sen_mapped))
#             print(sentence_frame_mapped_data.shape)
            count+=1
        except Exception as e:
            print(e)
            pass
        
print(len(sentence_frame_mapped_data))

C:\Users\Sashank\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


570


In [29]:
np.save('PCA_5feat_sentence_frame_mapped_l10.npy', sentence_frame_mapped_data)

In [20]:
# all the codes below this cell is just for testing and miscellaneous purposes, and can be ignored

In [19]:
for i in sentence_frame_mapped_data:
    print(i[0].shape,'--',i[1])

(127, 5) -- HELLO POSS-s:S NAME IS fs-BRUCE IX-1-s:S FROM NEW-YORK HERE IX-1-s:S 
(160, 5) -- RECENT GRADUATE GALLAUDET NOW IX-1-s:S SEARCH WORK NOW IX-1-s:S LIVE 
(226, 5) -- IN fs-BROOKLYN IN WHERE fs-CONEY ISLAND HAVE+ BROTHER ONE HEARING 
(213, 5) -- OLD TWENTY-SIX IX-1-s:1 HEARING BUT CAN SIGN IX-2-p:1,S TALL IX-1-s:1 
(156, 5) -- BROTHER SIX-FOUR IX-1-s:S SIX-ZERO IX-1-s:1 RECENT NOW GRADUATE SCHOOL IN 
(116, 5) -- HUNTER COLLEGE IX-1-s:S JUST GET #BS DEGREE IX-1-s:1 JUST WILL 
(87, 5) -- GET #BA SOON IX-1-s:S LIVE WITH PARENT YES HAVE BOTH 
(134, 5) -- OK MY BACKGROUND IX-1-s:S OK FINE IX-1-s:S POSS-s:S NAME fs-J 
(165, 5) -- EXCUSE IX-1-s:S POSS-s:S NAME fs-JEREMY IX-1-s:S GROW-UP HERE NEW-YORK IX-1-s:S 
(169, 5) -- GO DEAF SCHOOL PS47 IX-1-s:S HEARING FAMILY ONE DEAF BROTHER 
(134, 5) -- ONE HEARING SISTER IX-1-s:S YOUNG THIRD IX-1-s:S GO COLLEGE ONE 
(175, 5) -- YEAR IN GALLAUDET UNIVERSITY QUIT WORK FOR APPLE STORE #THEN 
(48, 5) -- WORK FOR FILM FILM EDITING #AT WHITNEY MUS

In [124]:
gloss_dir = 'Gloss/SIB01-story01-gloss.txt'
print(read_gloss(gloss_dir))

[['OK', '20', '28'], ['MY', '38', '40'], ['BACKGROUND', '44', '56'], ['IX-1-s:S', '62', '67'], ['OK', '76', '80'], ['FINE', '91', '94'], ['IX-1-s:S', '101', '117'], ['POSS-s:S', '121', '127'], ['NAME', '131', '135'], ['fs-J', '146', '154'], ['EXCUSE', '219', '222'], ['IX-1-s:S', '227', '228'], ['POSS-s:S', '242', '250'], ['NAME', '257', '265'], ['fs-JEREMY', '279', '304'], ['IX-1-s:S', '314', '328'], ['GROW-UP', '333', '338'], ['HERE', '342', '346'], ['NEW-YORK', '353', '356'], ['IX-1-s:S', '383', '384'], ['GO', '386', '395'], ['DEAF', '400', '404'], ['SCHOOL', '408', '415'], ['PS47', '422', '437'], ['IX-1-s:S', '483', '491'], ['HEARING', '499', '508'], ['FAMILY', '514', '521'], ['ONE', '527', '528'], ['DEAF', '532', '542'], ['BROTHER', '551', '555'], ['ONE', '582', '583'], ['HEARING', '588', '596'], ['SISTER', '605', '611'], ['IX-1-s:S', '622', '626'], ['YOUNG', '631', '635'], ['THIRD', '645', '652'], ['IX-1-s:S', '674', '677'], ['GO', '682', '693'], ['COLLEGE', '704', '711'], ['ONE',

In [139]:
for i in loaded_data:
    for key in i:
        if key == 'SIB01-story01-front_x264':
            print(key)

SIB01-story01-front_x264


In [35]:
loaded_data = np.load('key_points_5.npy')
print(loaded_data)

[{'SIA02-story01-front_x264': array([[-3235.6145597 ,  -127.95788802, -1405.83211229, -1116.6919313 ,
        -1145.27865466],
       [-3248.47008965,  -116.14910407, -1393.68410602, -1129.3161041 ,
        -1145.44649792],
       [-3237.67851661,  -127.85805667, -1365.59097089, -1132.49467098,
        -1153.66290886],
       ...,
       [-3293.69306899,  -134.55051818, -1400.36724605, -1139.74906403,
        -1192.19436685],
       [-3324.17609061,  -123.17064482, -1402.98807585, -1139.742507  ,
        -1195.2480783 ],
       [-3318.35126229,  -125.40226344, -1375.92315528, -1150.78510268,
        -1191.17466781]])}
 {'SIB01-story01-front_x264': array([[-3080.45372768,    41.78675729, -1377.92608453, -1432.90274095,
        -1091.68025204],
       [-3017.81671865,   -81.97932515, -1276.85955376, -1306.41625437,
        -1105.25749044],
       [-3076.39621442,    45.71354054, -1373.75708738, -1432.31947277,
        -1090.90368728],
       ...,
       [-3008.42695704,    81.3837115 , -

In [40]:
'Gloss/SIA02-story01-gloss.txt'.replace('Gloss/','').replace('.txt','').replace('gloss','front_x264')

SIA02-story01-front_x264
